In [1]:
import re
import time
import requests
import numpy as np

from bs4 import BeautifulSoup
from collections import Counter
from selenium import webdriver

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

from selenium.webdriver.common.by import By


In [2]:
url = 'https://www.usajobs.gov/Search/Results?k=data%20scientist'

In [3]:
## launch headless Edge instance
options = webdriver.EdgeOptions()
options.add_argument('--headless')

driver = webdriver.Edge(options = options)
driver.get(url)

## grab html from page
page = driver.page_source.encode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

## find total number of jobs found
job_count_ele = soup.find(id = 'page-info')
print(job_count_ele.prettify())


<h4 class="usajobs-search-controls__results-count" id="page-info">
 Viewing 1 – 25 of 181 jobs
</h4>



In [4]:
job_count = int(re.search(r'[0-9]+(?= jobs)', str(job_count_ele)).group())
print(job_count)

181


In [ ]:
## initiate variables for loop
df = pd.DataFrame(columns = ['job_title', 'job_url','job_summary', 'job_salary', 'job_location', 'job_quals'])
words = []
counter = 0
start = time.time()
i = 0
for ii in range(job_count // 25): 
    
    print("Starting")
    print(ii)
    page = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(page, 'html.parser')
    job_rows = soup.find_all('a', class_ = 'usajobs-search-result--core__title') 
    print("Number of jobs")
    print(len(job_rows))

    ## loop through rows, grabbing unique job posting url from each  
    for iii in range(len(job_rows)): # 
        
        ## grab url for unique job post
        job_link = re.search(r'(?<=href=\").*(?=\" id)', str(job_rows[iii])).group()
        print(job_link)
        j_url = 'https://www.usajobs.gov/'
        job_url = str(j_url + 
                      re.search(r'(?<=href=\").*(?=\" id)', 
                                str(job_rows[iii])).group()
                     )

        ## capture html from post page
        job_page = requests.get(job_url)
        job_soup = BeautifulSoup(job_page.content, 'html.parser')
        
        # grab job title
        job_title = job_soup.find('h1', class_ = 'usajobs-joa-banner__title')
        job_title = re.sub(r'\<.*\>', '', job_title.prettify())

        # grab job summary
        job_summary = job_soup.find('div', id = 'summary')
        job_summary = re.sub(r'\<.*\>', '', job_summary.prettify())
        
        # grab job description
        job_salary = job_soup.find('p', class_ = 'usajobs-joa-summary__salary salary-text-normal')
        job_salary = re.sub(r'\<.*\>', '', job_salary.prettify())
        
        # grab job location
        job_location = job_soup.find('div', class_ = "usajobs-joa-locations__body")
        job_location = re.sub(r'\<.*\>', '', job_location.prettify())
        
        # grab job qualifications
        job_quals = job_soup.find('div', id = "qualifications")
        job_quals = re.sub(r'\<.*\>', '', job_quals.prettify())
        
        job_duties = job_soup.find('div', id = "duties")
        job_duties = re.sub(r'\<.*\>', '', job_duties.prettify())
        
        
        
        ## move all variables to dataframe
        df.loc[i, 'job_title'] = job_title
        df.loc[i,'job_url'] = job_url
        df.loc[i,'job_summary'] = job_summary
        df.loc[i,'job_salary'] = job_salary
        df.loc[i,'job_location'] = job_location
        df.loc[i,'job_quals'] = job_quals
        df.loc[i,'job_duties'] = job_duties
        time.sleep(5)
        
        
        i = i + 1
        
    df2 = df
    df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
    df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
    df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
    df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
    df2.job_salary = df2.job_salary.str.replace('[^0-9]','-')
    df2.job_duties = df2.job_duties.str.replace('[^a-zA-Z0-9]',' ')
    df2.to_csv("data_scientist.csv")  
    
    button = driver.find_element(By.CLASS_NAME, 'usajobs-search-pagination__next-page')
    driver.execute_script('arguments[0].click()', button)
    time.sleep(2)

        

        
        

Starting
0
Number of jobs
25
/job/708958100
/job/708842100
/job/712013300
/job/711306400
/job/711306600
/job/709580700
/job/710110600
/job/710897600
/job/709571700
/job/710743700
/job/710744000
/job/711368100
/job/711702800
/job/711703400
/job/711338200
/job/711343400
/job/711774600
/job/709708600
/job/709708700
/job/707297000
/job/708597000
/job/708949100
/job/708949600
/job/711153200
/job/710310800


C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:76: FutureWarning:

Starting
1
Number of jobs
25
/job/712053500
/job/710200000
/job/710201500
/job/710983900
/job/711548000
/job/712057900
/job/690711800
/job/687971500
/job/688790200
/job/710921700
/job/710414900
/job/711871300
/job/711116800
/job/708343200
/job/708877400
/job/708878300
/job/708344100
/job/688827800
/job/711822700
/job/707624900
/job/711373200
/job/706077900
/job/704118800
/job/708058600
/job/693346100


C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:76: FutureWarning:

Starting
2
Number of jobs
25
/job/711403600
/job/710667300
/job/710667400
/job/710385500
/job/708690800
/job/709012600
/job/709528100
/job/702071600
/job/710640100
/job/710953200
/job/706514600
/job/703381400
/job/697689100
/job/709410700
/job/710805500
/job/710142100
/job/710933800
/job/709317300
/job/706746900
/job/681462700
/job/697044300
/job/707996000
/job/708544300
/job/709511000
/job/709516800


C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_title = df2.job_title.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_summary = df2.job_summary.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_location = df2.job_location.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.job_quals = df2.job_quals.str.replace('[^a-zA-Z0-9]', ' ')
C:\Users\kayle\AppData\Local\Temp\ipykernel_25068\1047686447.py:76: FutureWarning:

Starting
3
Number of jobs
25
/job/681462700
/job/711367300
/job/711428300
/job/711985100
/job/711985200


In [ ]:
len(df)

In [ ]:
df_save = df